In [142]:
# 3. DATA RETRIEVAL

In [2]:
#import key dependencies
import requests
import json
import pandas as pd
import numpy as np
import sqlite3

In [4]:
#get data as dataframe from endpoint
api_link="https://covid-api.com/api/"
data_key="data"
def get_data_from_endpoint(api_link,data_key,endpoint):
    response = requests.get(api_link+endpoint)
    json_load = json.loads(response.text)
    if type(json_load[data_key]) != list: #check for non-list. pandas.DataFrame throwing error while transforming, ValueError: If using all scalar values, you must pass an index
        json_load = [json_load[data_key]]
    else: json_load = json_load[data_key]
    df = pd.DataFrame(json_load)
    return df

In [5]:
#get data for regions
endpoint = "regions"
df_regions = get_data_from_endpoint(api_link,data_key,endpoint)
df_regions.head()

,iso,name
0,CHN,China
1,TWN,Taipei and environs
2,USA,US
3,JPN,Japan
4,THA,Thailand


In [9]:
#get data for provinces
list_df = []
list_iso = df_regions["iso"].tolist()
for ele in list_iso:
    endpoint = "provinces/"+ele
    df = get_data_from_endpoint(api_link,data_key,endpoint)
    list_df.append(df)
df_provinces = pd.concat(list_df)
df_provinces.reset_index(drop=True, inplace=True)
df_provinces.head()

,iso,name,province,lat,long
0,CHN,China,Anhui,31.8257,117.2264
1,CHN,China,Beijing,40.1824,116.4142
2,CHN,China,Chongqing,30.0572,107.8740
3,CHN,China,Fujian,26.0789,117.9874
4,CHN,China,Gansu,36.0611,103.8343
...,...,...,...,...,...
999,KIR,Kiribati,,-3.3704,-168.734
1000,PLW,Palau,,7.515,134.5825
1001,TON,Tonga,,-21.179000000000002,-175.1982
1002,NRU,Nauru,,-0.5228,166.9315


In [72]:
#get data for reports
endpoint = "reports"
df_reports = get_data_from_endpoint(api_link,data_key,endpoint)
df_reports.head()

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region
0,2023-03-09,209451,7896,0,0,0,0,2023-03-10 04:21:03,201555,0,0.0377,"{'iso': 'AFG', 'name': 'Afghanistan', 'provinc..."
1,2023-03-09,334457,3598,0,14,0,0,2023-03-10 04:21:03,330859,14,0.0108,"{'iso': 'ALB', 'name': 'Albania', 'province': ..."
2,2023-03-09,271496,6881,0,2,0,0,2023-03-10 04:21:03,264615,2,0.0253,"{'iso': 'DZA', 'name': 'Algeria', 'province': ..."
3,2023-03-09,47890,165,0,0,0,0,2023-03-10 04:21:03,47725,0,0.0034,"{'iso': 'AND', 'name': 'Andorra', 'province': ..."
4,2023-03-09,105288,1933,0,0,0,0,2023-03-10 04:21:03,103355,0,0.0184,"{'iso': 'AGO', 'name': 'Angola', 'province': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
786,2023-03-09,11526994,43186,0,0,0,0,2023-03-10 04:21:03,11483808,0,0.0037,"{'iso': 'VNM', 'name': 'Vietnam', 'province': ..."
787,2023-03-09,703228,5708,0,0,0,0,2023-03-10 04:21:03,697520,0,0.0081,"{'iso': 'PSE', 'name': 'West Bank and Gaza', '..."
788,2023-03-09,11945,2159,0,0,0,0,2023-03-10 04:21:03,9786,0,0.1807,"{'iso': 'YEM', 'name': 'Yemen', 'province': ''..."
789,2023-03-09,343135,4057,0,0,0,0,2023-03-10 04:21:03,339078,0,0.0118,"{'iso': 'ZMB', 'name': 'Zambia', 'province': '..."


In [79]:
#get data for reports_region
df_reports_region = pd.DataFrame(df_reports["region"].tolist())
df_reports_region.head()

,iso,name,province,lat,long,cities
0,AFG,Afghanistan,,33.9391,67.7100,[]
1,ALB,Albania,,41.1533,20.1683,[]
2,DZA,Algeria,,28.0339,1.6596,[]
3,AND,Andorra,,42.5063,1.5218,[]
4,AGO,Angola,,-11.2027,17.8739,[]
...,...,...,...,...,...,...
786,VNM,Vietnam,,14.0583,108.2772,[]
787,PSE,West Bank and Gaza,,31.9522,35.2332,[]
788,YEM,Yemen,,15.552726999999999,48.516388,[]
789,ZMB,Zambia,,-13.1339,27.8493,[]


In [127]:
#merge df_report with df_reports_region
merged_report = df_reports.join(df_reports_region)
merged_report

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region,iso,name,province,lat,long,cities
0,2023-03-09,209451,7896,0,0,0,0,2023-03-10 04:21:03,201555,0,0.0377,"{'iso': 'AFG', 'name': 'Afghanistan', 'provinc...",AFG,Afghanistan,,33.9391,67.7100,[]
1,2023-03-09,334457,3598,0,14,0,0,2023-03-10 04:21:03,330859,14,0.0108,"{'iso': 'ALB', 'name': 'Albania', 'province': ...",ALB,Albania,,41.1533,20.1683,[]
2,2023-03-09,271496,6881,0,2,0,0,2023-03-10 04:21:03,264615,2,0.0253,"{'iso': 'DZA', 'name': 'Algeria', 'province': ...",DZA,Algeria,,28.0339,1.6596,[]
3,2023-03-09,47890,165,0,0,0,0,2023-03-10 04:21:03,47725,0,0.0034,"{'iso': 'AND', 'name': 'Andorra', 'province': ...",AND,Andorra,,42.5063,1.5218,[]
4,2023-03-09,105288,1933,0,0,0,0,2023-03-10 04:21:03,103355,0,0.0184,"{'iso': 'AGO', 'name': 'Angola', 'province': '...",AGO,Angola,,-11.2027,17.8739,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2023-03-09,11526994,43186,0,0,0,0,2023-03-10 04:21:03,11483808,0,0.0037,"{'iso': 'VNM', 'name': 'Vietnam', 'province': ...",VNM,Vietnam,,14.0583,108.2772,[]
787,2023-03-09,703228,5708,0,0,0,0,2023-03-10 04:21:03,697520,0,0.0081,"{'iso': 'PSE', 'name': 'West Bank and Gaza', '...",PSE,West Bank and Gaza,,31.9522,35.2332,[]
788,2023-03-09,11945,2159,0,0,0,0,2023-03-10 04:21:03,9786,0,0.1807,"{'iso': 'YEM', 'name': 'Yemen', 'province': ''...",YEM,Yemen,,15.552726999999999,48.516388,[]
789,2023-03-09,343135,4057,0,0,0,0,2023-03-10 04:21:03,339078,0,0.0118,"{'iso': 'ZMB', 'name': 'Zambia', 'province': '...",ZMB,Zambia,,-13.1339,27.8493,[]


In [128]:
#drop column: region
merged_report.drop(columns="region",inplace=True)
merged_report.drop(columns="cities",inplace=True)
merged_report

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,iso,name,province,lat,long
0,2023-03-09,209451,7896,0,0,0,0,2023-03-10 04:21:03,201555,0,0.0377,AFG,Afghanistan,,33.9391,67.7100
1,2023-03-09,334457,3598,0,14,0,0,2023-03-10 04:21:03,330859,14,0.0108,ALB,Albania,,41.1533,20.1683
2,2023-03-09,271496,6881,0,2,0,0,2023-03-10 04:21:03,264615,2,0.0253,DZA,Algeria,,28.0339,1.6596
3,2023-03-09,47890,165,0,0,0,0,2023-03-10 04:21:03,47725,0,0.0034,AND,Andorra,,42.5063,1.5218
4,2023-03-09,105288,1933,0,0,0,0,2023-03-10 04:21:03,103355,0,0.0184,AGO,Angola,,-11.2027,17.8739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2023-03-09,11526994,43186,0,0,0,0,2023-03-10 04:21:03,11483808,0,0.0037,VNM,Vietnam,,14.0583,108.2772
787,2023-03-09,703228,5708,0,0,0,0,2023-03-10 04:21:03,697520,0,0.0081,PSE,West Bank and Gaza,,31.9522,35.2332
788,2023-03-09,11945,2159,0,0,0,0,2023-03-10 04:21:03,9786,0,0.1807,YEM,Yemen,,15.552726999999999,48.516388
789,2023-03-09,343135,4057,0,0,0,0,2023-03-10 04:21:03,339078,0,0.0118,ZMB,Zambia,,-13.1339,27.8493


In [11]:
#get data for reports/total
endpoint = "reports/total"
df_reports_total = get_data_from_endpoint(api_link,data_key,endpoint)
df_reports_total

,date,last_update,confirmed,confirmed_diff,deaths,deaths_diff,recovered,recovered_diff,active,active_diff,fatality_rate
0,2023-03-09,2023-03-10 04:21:03,676544789,194101,6881737,1854,0,0,669663052,192247,0.0102


In [12]:
# Data integration - export data to sqlite db
sqlite_db = r"D:\git_repo_IO_mini_project1\IOD_mini_project1\preprocess_db.sqlite" # rename root directory to your own directory
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [121]:
#add df_regions to db
df_regions.to_sql('regions', con = conn, if_exists = 'replace')

219

In [125]:
#add df_provinces to db
df_provinces.to_sql('provinces', con = conn, if_exists = 'replace')

1004

In [129]:
#add merged_report to db
merged_report.to_sql('merged_report', con = conn, if_exists = 'replace')

791

In [130]:
#add df_reports_total to db
df_reports_total.to_sql('reports_total', con = conn, if_exists = 'replace')

1

In [ ]:
#test query to fetch data from db
c.execute("SELECT * FROM regions")
c.fetchall()